- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

# Embeddings(Text, Audio) Session 1 
- 각세션으로 확대시켜야 함

In [23]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd


In [24]:
file_name = 'Sess01_eval.csv'
df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation['Emotion'], df_annotation['Valence'], df_annotation['Arousal']
file_lst = df_annotation[' .1']
file_lst

0      Sess01_script01_User002M_001
1      Sess01_script01_User002M_002
2      Sess01_script01_User002M_003
3      Sess01_script01_User002M_004
4      Sess01_script01_User001F_001
                   ...             
306    Sess01_script06_User001F_016
307    Sess01_script06_User002M_041
308    Sess01_script06_User002M_042
309    Sess01_script06_User002M_043
310    Sess01_script06_User001F_017
Name:  .1, Length: 311, dtype: object

## Audio Embedding

In [25]:
wav_file_lst = []
for i in file_lst:
    wav_file_lst.append('./org_KEMDy20/Session01/' + i + '.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.wav']

In [26]:
def get_file_names(file_lst):
    file_name_lst = []
    for i in file_lst:
        i = i.split('/')[-1]
        i = i.split('.')[-2]
        file_name_lst.append(i)
    return file_name_lst
    

In [27]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
dataset[0]["wav"]

{'path': './org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 'array': array([ 0.0526123 ,  0.05267334,  0.05197144, ..., -0.00387573,
        -0.00244141, -0.00137329], dtype=float32),
 'sampling_rate': 16000}

In [28]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav_cuda = Data2VecAudioModel.from_pretrained(check_point_wav_model).cuda()
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from

In [29]:
dataset[0]['wav']['array']

array([ 0.0526123 ,  0.05267334,  0.05197144, ..., -0.00387573,
       -0.00244141, -0.00137329], dtype=float32)

In [30]:
sampling_rate = dataset['wav'][0]['sampling_rate']
sampling_rate

16000

In [31]:
for i in range(3):
    input = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = True, max_length = 50)
    with torch.no_grad():
        output = model_wav(**input)
output['last_hidden_state'].shape

torch.Size([1, 410, 768])

In [32]:
def wav_embeddings_func(wav_file_lst, processor, model):
    dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
    wav_embeddings = []
    for i in range(len(dataset)): 
        inputs = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = True, max_length=400)
        with torch.no_grad():
            output = model_wav(**inputs)
        wav_embeddings.append(output['last_hidden_state'])
    return wav_embeddings

## Text Embedding


- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [33]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import re


In [34]:
# text data loading
txt_file_lst = []
for i in file_lst:
    txt_file_lst.append('./org_KEMDy20/Session01/' + i + '.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.txt']

In [35]:
# 참고: https://huggingface.co/gogamza/kobart-base-v2
# 참고: https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need

# model = AutoModel.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
# tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
checkpoint = 'BM-K/KoSimCSE-bert-multitask'
model_txt = AutoModel.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'


In [36]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []

    for i in txt_file_lst:    
        f = open(i, 'r')
        line = f.readline()
        # line = re.sub('l/\n|c/|b/|n/|l/|u/', '', line)
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
    start_index = np.random.randint(1, high = len(txt_file_lst)) - 10
    print(sentences[start_index:start_index+10])
    
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    txt_embeddings, _ = model_txt(**inputs, return_dict=False)
    return txt_embeddings
    

# 데이터셋 만들기

In [37]:
# # wav file embedding
# wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav)
# wav_embeddings[:3]

In [38]:
# # txt file embedding
# txt_embeddings = text_embeddings_func(txt_file_lst,tokenizer, model_txt)
# txt_embeddings[:3]

In [39]:
# file_name_lst = get_file_names(wav_file_lst)
# file_name_lst[:5]

In [40]:
# # annotation data 가져오기
# df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
# df_annotation.Arousal[:3], df_annotation.Emotion[:3], df_annotation.Valence[:3]

In [41]:
# print(len(txt_embeddings), len(wav_embeddings), len(file_name_lst))

In [42]:
def build_dataset(file_name_lst, 
                  text_embeddings, 
                  wav_embeddings, 
                  label_emotion,
                  label_arousal,
                  label_valence, 
                  session_name = 'Session01'):
    dataset = {'file_names': file_name_lst, 
         'text_embeddings': text_embeddings, 
         'wav_embeddings':wav_embeddings,
         'Emotion':label_emotion,
         'Arousal':label_arousal,
         'Valence':label_valence}
    return dataset

In [43]:
session_num_lst = []
annot_num_lst = []
for i in range(40):
    if i <= 8:
        session_num_lst.append('Session0' + str(i+1))
        annot_num_lst.append('Sess0' + str(i+1) + '_eval.csv')
    else:
        session_num_lst.append('Session' + str(i+1))
        annot_num_lst.append('Sess' + str(i+1) + '_eval.csv')
session_num_lst[-5:], annot_num_lst[-5:]

(['Session36', 'Session37', 'Session38', 'Session39', 'Session40'],
 ['Sess36_eval.csv',
  'Sess37_eval.csv',
  'Sess38_eval.csv',
  'Sess39_eval.csv',
  'Sess40_eval.csv'])

In [44]:
# df_annotation = pd.read_csv('org_KEMDy20/annotation/' + annot, skiprows=1)
# df_annotation

In [45]:
dataset = {}
for annot, session in zip(annot_num_lst[:3], session_num_lst[:3]):
    
    df_annotation = pd.read_csv('org_KEMDy20/annotation/' + annot, skiprows=1)
    file_name_lst = df_annotation[' .1']
    
    txt_file_lst = []
    wav_file_lst = []
    for file_name in file_name_lst:
        txt_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.txt')
        wav_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.wav')
    
    txt_embeddings = text_embeddings_func(txt_file_lst, tokenizer, model_txt)
    wav_embeddings = wav_embeddings_func(wav_file_lst, processor, model_wav_cuda)
    
    dataset[session] = build_dataset(file_name_lst, 
                txt_embeddings, 
                wav_embeddings,  
                df_annotation.Arousal, 
                df_annotation.Emotion, 
                df_annotation.Valence)

['c/ 기차를 타고.', 'c/ 근데 그때가 이제 처음 방을 구하러 내려온 날이었거든.', '근데 처음에 이제 기차를 서울에서 타고 오는데 친해진 옆자리 사람도 같 원광대 오는 사람인 거야.', 'c/ 바로 옆에 앉 u/ .', 'c/ 이제 그게 익산 가까이 와서 좀 알게 됐는데 그 사람도 방을 보러 왔 온다고 하더라고.', '그래서 같이 어 그러면 같이 방을 볼래요 하면서 같이 방을 보다가 그 결국은 둘이 같이 룸메를 해서 하숙집에 들어갔어.', '그래갖고 근데 지 난 그 둘이 되게 친해졌을 거라고 생각을 했거든?', 'c/ 근데 진짜 방만 공유하고 서로 아무 대화도 안 한다는 거야. l/', '심지어 같은 원룸에서 같이 지내는데.', '그냥 이불을 깔고 같이 지내는데 아무 그거를 안 얘기를 거의 안 한다고 하더라고, 어 왔어 정도? l/']
['c/ 그래서 언니가 지금 딴 데 가있어서 그래서 엄마가 아빠 술 먹고 좀 좀 만취한 거 같으면 아예 다른 방에서 주무세요.', 'c/ 소리가 너무 커 가지고 n/ 저 방과 방 끝 끝에 있는데도 b/ 여기 끝에서 아빠가 아 오늘 아빠 코골이 많이 하네 좀 알 정 도로 많이 들리더라고요.', '아 딱 인제 제 친구가 그 정도의 소리였을 거 같애요, 그때.', 'c/ 진짜 엄청 컸거든요.', '너무 코골이가 심하고 제 친구도 코골이 진짜 심한 애가 기숙사 그니까 두 명 있었거든요.', '둘 다 기숙사 살았는데 룸메가 2주?', '진짜 길면 한 달 살다가 다 나갔어요.', '도저히 얘 코골이 너무 심해서 같이 못 살겠다고 다른 방 해달 해달라 그래가지구 b/ 중간에 탈주 엄청 하더라고요.', 'c/ 도저히 못 자겠다고.', '아 근데 진짜 코골이는 너무 심하면 좀 그래서 근데 또 l/ 신 신기한게 코고는 친구들끼리는 엄청 잘 자더라고요, 같이 l/ u/ 어 b/ 어 어 너무 u/ 하면서 옆에 있는 코 안 고는 애들 u/ 야 너희들 코 엄청 곤다고 하면서 어 그래 골았어 내가 이러면서.']
['아 그니까 나한테는 그 지갑 

In [49]:
dataset.keys()

dict_keys(['Session01', 'Session02', 'Session03'])

In [72]:
# length test
for i in dataset['Session01'].keys():
    print(len(dataset['Session01'][i]))


311
311
311
311
311
311


# save dataset

In [73]:
import pickle

# save data set
with open('./model/data/lou_dataset_1_3.pkl', 'wb')as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

In [74]:
with open('./model/data/lou_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])